In [15]:
import tarfile
import pandas as pd
import random
import io
import pickle
import os

# Extract 100 random daily measure locations
### Store individually in 'lamah_100_dataframes.csv'
### Store concatinated in 'lamah_100_concatenated.csv'


In [16]:
# Path to your 2_LamaH-CE_daily.tar.gz file
tar_path = "./2_LamaH-CE_daily.tar.gz"
output_dir = "./data"

# Make sure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Open the tar.gz file
with tarfile.open(tar_path, "r:gz") as tar:
    # Get all CSV file members "A_basins_total_upstrm -> 2_timeseries -> daily -> *.csv"
    csv_files = [
        member for member in tar.getmembers()
        if member.name.startswith("A_basins_total_upstrm/2_timeseries/daily/")
        and member.name.endswith(".csv")
    ]
    
    # Randomly select 100 CSV files
    selected_files = random.sample(csv_files, 100)
    
    print(f"Selected {len(selected_files)} files.")
    
    first_df = None
    
    for member in selected_files:
        # Extract file content as bytes
        f = tar.extractfile(member)
        if f is not None:
            # LamaH files are semicolon-separated
            df = pd.read_csv(io.BytesIO(f.read()), sep=";", encoding="utf-8")
            
            # Save each DataFrame as its own CSV in ./data/
            filename_only = os.path.basename(member.name)   # e.g. "ID_163.csv"
            out_path = os.path.join(output_dir, filename_only)
            df.to_csv(out_path, index=False)
            
            # Keep first df just to show a sample
            if first_df is None:
                first_df = df.copy()

print("Saved 100 separate CSV files to ./data/")
if first_df is not None:
    print("Sample DataFrame (first file):")
    print(first_df.head())

# Optional: attributes for later use
if first_df is not None:
    attributes = list(first_df.columns)
    print("\nAttributes:", attributes)

Selected 100 files.
Saved 100 separate CSV files to ./data/
Sample DataFrame (first file):
   YYYY  MM  DD  DOY  2m_temp_max  2m_temp_mean  2m_temp_min  2m_dp_temp_max  \
0  1981   1   1    1          3.3           1.1         -1.4            -0.2   
1  1981   1   2    2          1.2          -0.5         -1.9            -1.0   
2  1981   1   3    3          4.5           3.0          1.4             1.8   
3  1981   1   4    4          2.5           1.0         -1.2             0.3   
4  1981   1   5    5         -1.7          -3.3         -5.2            -3.6   

   2m_dp_temp_mean  2m_dp_temp_min  ...   swe  surf_net_solar_rad_max  \
0             -3.3            -7.3  ...  1.12                     169   
1             -4.3            -7.2  ...  2.38                     175   
2              0.6            -1.4  ...  6.24                      98   
3             -1.3            -2.8  ...  4.83                     175   
4             -5.4            -7.4  ...  7.60                  

### How to load data from now on only from pickle file

In [17]:
csv_dir = "./data"

# List all CSVs in the folder
csv_files = sorted([f for f in os.listdir(csv_dir) if f.endswith(".csv")])

print("Found CSV files:", len(csv_files))
print("First two files:", csv_files[:2])

# Load the first two CSVs
df1 = pd.read_csv(os.path.join(csv_dir, csv_files[0]))
df2 = pd.read_csv(os.path.join(csv_dir, csv_files[1]))

print("\nFirst CSV file:", csv_files[0])
print(df1.head())

print("\nSecond CSV file:", csv_files[1])
print(df2.head())

Found CSV files: 100
First two files: ['ID_10.csv', 'ID_104.csv']

First CSV file: ID_10.csv
   YYYY  MM  DD  DOY  2m_temp_max  2m_temp_mean  2m_temp_min  2m_dp_temp_max  \
0  1981   1   1    1         -2.5          -5.4         -9.4            -4.2   
1  1981   1   2    2         -2.9          -4.5         -5.9            -4.6   
2  1981   1   3    3          1.8          -0.7         -2.8             0.1   
3  1981   1   4    4          1.5          -1.3         -4.7             0.2   
4  1981   1   5    5         -5.4          -6.3         -7.2            -7.1   

   2m_dp_temp_mean  2m_dp_temp_min  ...     swe  surf_net_solar_rad_max  \
0             -8.2           -13.5  ...  173.91                      94   
1             -6.3            -7.3  ...  186.53                      61   
2             -3.1            -5.0  ...  200.98                      68   
3             -2.1            -5.9  ...  229.64                      24   
4             -7.9            -9.0  ...  254.30    

| Attribute               | Description                                                                                                                |
|-------------------------|----------------------------------------------------------------------------------------------------------------------------|
| YYYY                    | Year of observation (e.g., 1981)                                                                                           |
| MM                      | Month (1 to 12)                                                                                                            |
| DD                      | Day of month (1 to 31)                                                                                                     |
| DOY                     | Day of year (1 to 365/366); allows for seasonal analysis                                                                  |
| 2m_temp_max             | Maximum air temperature at 2 meters above ground (°C)                                                                      |
| 2m_temp_mean            | Mean air temperature at 2 meters above ground (°C)                                                                         |
| 2m_temp_min             | Minimum air temperature at 2 meters above ground (°C)                                                                      |
| 2m_dp_temp_max          | Maximum dew point temperature at 2 meters (°C); temperature at which air becomes saturated                                  |
| 2m_dp_temp_mean         | Mean dew point temperature at 2 meters (°C)                                                                                 |
| 2m_dp_temp_min          | Minimum dew point temperature at 2 meters (°C)                                                                             |
| 10m_wind_u              | East-west wind speed component at 10 meters (m/s)                                                                           |
| 10m_wind_v              | North-south wind speed component at 10 meters (m/s)                                                                        |
| fcst_alb                | Forecasted surface albedo (reflectivity; unitless or percent)                                                               |
| lai_high_veg            | Leaf Area Index for high vegetation; proxy for vegetation cover (dimensionless)                                             |
| lai_low_veg             | Leaf Area Index for low vegetation (dimensionless)                                                                          |
| swe                     | Snow Water Equivalent (mm); total water content in snowpack                                                                |
| surf_net_solar_rad_max  | Maximum net solar radiation at surface (W/m²); key for surface heating and energy balance                                    |
| surf_net_solar_rad_mean | Mean net solar radiation at surface (W/m²)                                                                                  |
| surf_net_therm_rad_max  | Maximum net thermal (infrared) radiation at surface (W/m²); affects nighttime cooling                                        |
| surf_net_therm_rad_mean | Mean net thermal radiation at surface (W/m²)                                                                                |
| surf_press              | Surface atmospheric pressure (hPa)                                                                                           |
| total_et                | Total evapotranspiration (mm); water flux from land/vegetation to atmosphere                                                |
| prec                    | Precipitation (mm); daily total                                                                                             |
| volsw_123               | Volumetric soil water content in soil layers 1+2+3 (mm or %)                                                               |
| volsw_4                 | Volumetric soil water content in soil layer 4 (mm or %)                                                                     |
